In [ ]:
# Install dependencies (run once in Colab)
!pip install music21 numpy

# Import libraries
from music21 import converter, stream, note
import numpy as np
import random

# New Function: Check for Overlapping Notes
def check_overlapping_notes(midi_file_path):
    # Load MIDI file using converter
    midi_stream = converter.parse(midi_file_path)

    # Get all notes with their start and end times
    notes_with_times = []
    for element in midi_stream.flat.notes:
        if isinstance(element, note.Note):  # Only single notes
            start_time = element.offset  # When the note starts (in quarter notes)
            duration = element.quarterLength  # How long it lasts
            end_time = start_time + duration
            notes_with_times.append((str(element.pitch), start_time, end_time))

    # Check for overlaps
    overlapping = False
    for i in range(len(notes_with_times) - 1):
        current_note, start1, end1 = notes_with_times[i]
        next_note, start2, end2 = notes_with_times[i + 1]

        # If the next note starts before the current note ends, they overlap
        if start2 < end1:
            print(f"Overlap detected: {current_note} (ends {end1}) overlaps with {next_note} (starts {start2})")
            overlapping = True

    if not overlapping:
        print("No overlapping notes detected. File is monophonic.")
    return not overlapping, notes_with_times

# Parse MIDI to Notes (Modified to Use Timed Notes)
def parse_midi_to_notes(notes_with_times):
    # Extract just the note names from the timed list
    notes = [note for note, _, _ in notes_with_times]
    return notes

# Build Transition Matrix
def build_transition_matrix(notes):
    unique_notes = sorted(set(notes))
    num_states = len(unique_notes)
    note_to_index = {note: idx for idx, note in enumerate(unique_notes)}
    transition_matrix = np.zeros((num_states, num_states))

    for i in range(len(notes) - 1):
        current_note = notes[i]
        next_note = notes[i + 1]
        transition_matrix[note_to_index[current_note]][note_to_index[next_note]] += 1

    for i in range(num_states):
        row_sum = np.sum(transition_matrix[i])
        if row_sum > 0:
            transition_matrix[i] /= row_sum

    return transition_matrix, unique_notes, note_to_index

# Generate Melody
def generate_melody(transition_matrix, unique_notes, note_to_index, length=10, start_note=None):
    if start_note is None or start_note not in unique_notes:
        current_note = random.choice(unique_notes)
    else:
        current_note = start_note

    melody = [current_note]
    current_idx = note_to_index[current_note]

    for _ in range(length - 1):
        probs = transition_matrix[current_idx]
        if np.sum(probs) == 0:
            next_note = random.choice(unique_notes)
        else:
            next_idx = np.random.choice(len(unique_notes), p=probs)
            next_note = unique_notes[next_idx]
        melody.append(next_note)
        current_idx = note_to_index[next_note]

    return melody

# Save Melody to MIDI
def save_melody_to_midi(melody, output_file='mm_generated_melody.mid'):
    s = stream.Stream()
    for n in melody:
        new_note = note.Note(n)
        new_note.quarterLength = 1
        s.append(new_note)
    s.write('midi', fp=output_file)
    print(f"Saved melody to {output_file}")

# Main Function
def main():
    midi_file_path = 'FILE PATH'  # Replace with your exact path

    # Check for overlapping notes
    is_monophonic, notes_with_times = check_overlapping_notes(midi_file_path)
    if not is_monophonic:
        print("Warning: File is polyphonic. Results may not be ideal for a basic MM.")

    # Parse notes
    notes = parse_midi_to_notes(notes_with_times)
    print("Original notes:", notes[:10], "...")

    # Build transition matrix
    transition_matrix, unique_notes, note_to_index = build_transition_matrix(notes)
    print("Unique notes (states):", unique_notes)
    print("Transition matrix shape:", transition_matrix)

    # Generate melody
    new_melody = generate_melody(transition_matrix, unique_notes, note_to_index, length=15, start_note=notes[0])
    print("Generated melody:", new_melody)

    # Save to MIDI
    save_melody_to_midi(new_melody, 'mm_generated_melody.mid')

if __name__ == "__main__":
    main()